# Part 3: Validation

In [ ]:
%%capture
%pip install python-terrier

import pyterrier as pt
from pyterrier.measures import *
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


from my_utils import query_similar_words
from my_utils import preprocess

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64/"

if not pt.started():
  pt.init()

In [ ]:
def create_index(docs_df, mode="both", indexer_path="./pd_index"):

    pd_indexer = pt.DFIndexer(indexer_path, remove_stopwords=False, overwrite=True)
    pd_indexer.setProperty("termpipelines", "")
    #pd_indexer.setProperty("metaindex.compressed.reverse.allow.duplicates", "False")

    if mode == "title":
        index_ref = pd_indexer.index(docs_df["title"], docs_df["docno"])
    elif mode == "abstract":
        index_ref = pd_indexer.index(docs_df["abstract"], docs_df["docno"])
    else:  # both or else
        index_ref = pd_indexer.index(docs_df["title"], docs_df["abstract"], docs_df["docno"])

    return index_ref

def index_model(index, model="TF_IDF"):
    """
    Create and return an indexing model.
    http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html
    Model can be such as:
    "TF_IDF", "PL2", "DLH",
    "Hiemstra_LM", "Dirichlet_LM",
    "BM25", etc
    """
    list_model = ["TF_IDF", "PL2", "DLH", "Hiemstra_LM", "Dirichlet_LM", "BM25"]

    indexing_model = pt.BatchRetrieve(index, wmodel=model)
    return indexing_model

def preprocess_queries(queries, augemented=False, mv_model=None):
    queries_preprocessed = queries.copy()
    for index, row in tqdm(queries.iterrows(), total=len(queries)):
        pre_processed_title = " ".join(preprocess(row["title"]))
        pre_processed_description = " ".join(preprocess(row["description"]))
        pre_processed_narrative = " ".join(preprocess(row["narrative"]))

        if augemented:
            pre_processed_title = query_similar_words(pre_processed_title, mv_model)
            pre_processed_description = query_similar_words(pre_processed_description, mv_model)
            pre_processed_narrative = query_similar_words(pre_processed_narrative, mv_model)

        queries_preprocessed.loc[index, "title"] = pre_processed_title
        queries_preprocessed.loc[index, "description"] = pre_processed_description
        queries_preprocessed.loc[index, "narrative"] = pre_processed_narrative

    if augemented:
        queries_preprocessed.to_pickle("data/queries_processed_aug.pkl")
    else:
        queries_preprocessed.to_pickle("data/queries_processed.pkl")

    return queries_preprocessed

def expirement(indexed_docs, preprocessed_query, qrels):
    tfidf = index_model(indexed_docs, model="TF_IDF")
    bm25 = index_model(indexed_docs, model="BM25")
    dirichlet_lm = index_model(indexed_docs, model="DirichletLM")

    exp = pt.Experiment(
        [tfidf, bm25, dirichlet_lm],
        preprocessed_query,
        qrels,
        eval_metrics=[P@5, P@10, nDCG@10, RR(rel=2)],
        names=["TF_IDF", "BM25", "DirichletLM"]
    )
    return exp

## Experiments
 - preprocessing: {all languages, only english}
 - topics: {title, description, narrative}
 - indexing: {title, abstract, both}
 - models: {tf_idf, bm25, dirichlet_lm}

In [ ]:
%%capture

# read preprocessed en-documents
en_docs = pd.read_pickle('./data/en_docs_processed.pkl')
en_docs.rename(columns={"doc_id":"docno"}, inplace=True)

# index en-documents by (title, abstract, both)
index_ref_en_docs_title = create_index(en_docs, mode="title", indexer_path="./indexes/en/title")
index_en_docs_title = pt.IndexFactory.of(index_ref_en_docs_title)

index_ref_en_docs_abstract = create_index(en_docs, mode="abstract", indexer_path="./indexes/en/abstract")
index_en_docs_abstract = pt.IndexFactory.of(index_ref_en_docs_abstract)

index_ref_en_docs_both = create_index(en_docs, mode="both", indexer_path="./indexes/en/both")
index_en_docs_both = pt.IndexFactory.of(index_ref_en_docs_both)

In [ ]:
%%capture
# read preprocessed documents
docs = pd.read_pickle('./data/docs_processed.pkl')
docs.rename(columns={"doc_id":"docno"}, inplace=True)

# index documents by (title, abstract, both)
index_ref_docs_title = create_index(docs, mode="title", indexer_path="./indexes/all/title")
index_docs_title = pt.IndexFactory.of(index_ref_docs_title)

index_ref_docs_abstract = create_index(docs, mode="abstract", indexer_path="./indexes/all/abstract")
index_docs_abstract = pt.IndexFactory.of(index_ref_docs_abstract)

index_ref_docs_both = create_index(docs, mode="both", indexer_path="./indexes/all/both")
index_docs_both = pt.IndexFactory.of(index_ref_docs_title)

In [ ]:
AUGMENTED = True
if AUGMENTED:
    from gensim.models import Word2Vec
    mv_model = Word2Vec.load("data/word2vec.model")

# preprocess queries by (title, description, narrative)
import ir_datasets
dataset = ir_datasets.load("cord19/trec-covid")
queries = pd.DataFrame(dataset.queries_iter())
queries = preprocess_queries(queries, augemented=AUGMENTED, mv_model=mv_model)
#queries = preprocess_queries(queries)
display(queries.head(5))

pq_title = queries[['query_id', 'title']]
pq_title = pq_title.rename(columns={'query_id':'qid', 'title':'query'})
display(pq_title.head(5))

pq_desc = queries[['query_id', 'description']]
pq_desc = pq_desc.rename(columns={'query_id':'qid', 'description':'query'})
display(pq_desc.head(5))

pq_narr = queries[['query_id', 'narrative']]
pq_narr = pq_narr.rename(columns={'query_id':'qid', 'narrative':'query'})
display(pq_narr.head(5))

In [ ]:
# load trec-covid dataset
cord19 = pt.get_dataset("irds:cord19/trec-covid")
qrels = cord19.get_qrels()

In [ ]:
# en_docs {title, abstract, both} with query_title
exp1 = expirement(index_en_docs_title, pq_title, qrels)
display(exp1)

exp2 = expirement(index_en_docs_abstract, pq_title, qrels)
display(exp2)

exp3 = expirement(index_en_docs_both, pq_title, qrels)
display(exp3)

# en_docs {title, abstract, both} with query_description
exp4 = expirement(index_en_docs_title, pq_desc, qrels)
display(exp4)

exp5 = expirement(index_en_docs_abstract, pq_desc, qrels)
display(exp5)

exp6 = expirement(index_en_docs_both, pq_desc, qrels)
display(exp6)

# en_docs {title, abstract, both} with query_narrative
exp7 = expirement(index_en_docs_title, pq_narr, qrels)
display(exp7)

exp8 = expirement(index_en_docs_abstract, pq_narr, qrels)
display(exp8)

exp9 = expirement(index_en_docs_both, pq_narr, qrels)
display(exp9)

In [ ]:
# docs {title, abstract, both} with query_title
exp1 = expirement(index_docs_title, pq_title, qrels)
display(exp1)

exp2 = expirement(index_docs_abstract, pq_title, qrels)
display(exp2)

exp3 = expirement(index_docs_both, pq_title, qrels)
display(exp3)

# docs {title, abstract, both} with query_description
exp4 = expirement(index_docs_title, pq_desc, qrels)
display(exp4)

exp5 = expirement(index_docs_abstract, pq_desc, qrels)
display(exp5)

exp6 = expirement(index_docs_both, pq_desc, qrels)
display(exp6)

# docs {title, abstract, both} with query_narrative
exp7 = expirement(index_docs_title, pq_narr, qrels)
display(exp7)

exp8 = expirement(index_docs_abstract, pq_narr, qrels)
display(exp8)

exp9 = expirement(index_docs_both, pq_narr, qrels)
display(exp9)

# Pairwise Similarity of Queries
# - TF-IDF
# - DOC2VEC

Write a function that measures the pairwise similarity of the provided 
queries. Create a list of similar queries and a word-cloud of their qids.

In [ ]:
%%capture
%pip install gensim

import pandas as pd
import numpy as np
import wordcloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

def compute_freqs(queries, labels):
    freqs = {}
    for i in range(max(labels)):
        # print(f"cluster {i:2} {queries.loc[labels==i]['title'].values}")

        similar_queries = queries.loc[labels==i]['query_id'].values
        for qid in similar_queries:
            freqs[qid] = len(similar_queries)

    return freqs

def print_clustered_queries(queries, labels):
    for i in range(max(labels)):
        print(f"cluster {i:2} {queries.loc[labels==i]['description'].values}")

In [ ]:
if AUGMENTED:
    queries = pd.read_pickle('./data/queries_processed_aug.pkl')
else:
    queries = pd.read_pickle('./data/queries_processed.pkl')

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(queries['description']).toarray()

cluster = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=0.9)
labels = cluster.fit_predict(vectors)

print_clustered_queries(queries, labels)

In [ ]:
wordcloud.WordCloud().fit_words(compute_freqs(queries, labels)).to_image()

In [ ]:
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(queries['description'])]
doc_model = Doc2Vec(tagged_data, window=5, seed=1)
d2v_doc_embeddings = np.array([doc_model.infer_vector(doc.split()) for doc in queries['description']])
print(d2v_doc_embeddings.shape)

cluster = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=0.95)
labels = cluster.fit_predict(d2v_doc_embeddings)

print_clustered_queries(queries, labels)

In [ ]:
wordcloud.WordCloud().fit_words(compute_freqs(queries, labels)).to_image()

In [ ]:
similar_queries = queries[labels == 0][['query_id', 'description']]
similar_queries = similar_queries.rename(columns={'query_id':'qid', 'description':'query'})
similar_queries

In [ ]:
tfidf = index_model(index_en_docs_abstract, model="TF_IDF")
bm25 = index_model(index_en_docs_abstract, model="BM25")
dirichlet_lm = index_model(index_en_docs_abstract, model="DirichletLM")

exp = pt.Experiment(
    [bm25],
    similar_queries,
    qrels,
    perquery=True,
    eval_metrics=[P@5, P@10, nDCG@10, RR(rel=2)],
    names=["BM25"]
)
display(exp)